## Filter
One of the most powerful parameters when defining a search is the ability to filter the results.  While a Query is more of an unstructured approach to search for text within documents, a Filter provides a well-defined structured approach when we know which properties we want to restrict our matching documents.

The following examples will be using some of the popular properties as outlined within the "Properties & Metadata" notebook.  In addition, the selection of these properties was determined based on the discovery techniques used such as metadata and debugging.

In [1]:
import refinitiv.dataplatform as rdp
import pandas as pd
import datetime as dt
import dateutil.relativedelta

rdp.open_desktop_session('Your API Key here')

In [2]:
pd.set_option('display.max_colwidth', 140)
rdp.__version__

'1.0.0a7.post1'

#### Example - Query vs Filter
As you become more familiar with Search, you will realize there are a number of different paths you can take to arrive at the same destination.

In [3]:
# Retrieve the document(s) for a specific ISIN using a Query.
rdp.search(
    view = rdp.SearchViews.Instruments,
    query = 'US046353AQ14',
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
)

,DocumentTitle,ISIN,PermID,SEDOL,CUSIP,RIC
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,46645702869,BF4SNM7,046353AQ1,046353AQ1=


In [4]:
# Retrieve the document(s) for the same ISIN using a Filter
rdp.search(
    view = rdp.SearchViews.Instruments,
    filter = "ISIN eq 'US046353AQ14'",
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
)

,DocumentTitle,ISIN,PermID,SEDOL,CUSIP,RIC
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,46645702869,BF4SNM7,046353AQ1,046353AQ1=


In [5]:
# If you are provided a code but are not sure if the code is an ISIN, SEDOL, CUSIP etc, 
# try using the 'CodeResolutionBase' property.
rdp.search(
    view = rdp.SearchViews.Instruments,
    filter = "CodeResolutionBase eq '046353AQ1'",   # I'm using the CUSIP for the same instrument
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
)

,DocumentTitle,ISIN,PermID,SEDOL,CUSIP,RIC
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,46645702869,BF4SNM7,046353AQ1,046353AQ1=


#### Example - Apple Equities

In [6]:
# Within the view (EquityQuotes), select all documents where the term 'AAPL' is within the ticker symbol.  For each, 
# ensure the type of asset is an 'equity' and that each asset is in an active state.  I chose to filter out the 
# AssetState of 'DC' (De-activated) as my criteria.
#
# Note: An alternative to "AssetType eq 'equity'" could be to use: "RCSAssetCategoryLeaf eq 'Ordinary Share'"
rdp.search(
    view = rdp.SearchViews.EquityQuotes,
    filter = "AssetType eq 'equity' and AssetState ne 'DC' and TickerSymbol eq 'AAPL'",
    select = "RIC, CompositeDescriptiveName, ExchangeCode, ExchangeName, AssetType"
)

,AssetType,CompositeDescriptiveName,RIC,ExchangeName,ExchangeCode
0,[EQUITY],Apple Ord Shs APPLE ORD,AAPL.O,NASDAQ Global Select Consolidated,NSQ
1,[EQUITY],Apple Ord Shs APPLE ORD APPLE INC,AAPLEUR.xbo,MiFID Composite Cross Market Service,XBO
2,[EQUITY],Apple Ord Shs APPLE ORD,AAPL.OQ,NASDAQ Stock Exchange Global Select Market,NSM
3,[EQUITY],Apple Ord Shs APPLE ORD,AAPL.DG,Cboe EDGX Exchange - Nasdaq Global Select Market,GCD
4,[EQUITY],Apple Ord Shs APPLE ORD APPLE INC,AAPL.Z,Cboe BZX Exchange - Nasdaq Global Select Market,GCZ
5,[EQUITY],Apple Ord Shs APPLE ORD,AAPL.B,Boston SE when trading NASDAQ Global Select Market,GSB
6,[EQUITY],Apple Ord Shs APPLE ORD,AAPLE.MI,Milan Stock Exchange,MIL
7,[EQUITY],Apple Ord Shs APPLE ORD APPLE INC,AAPL.ZY,Cboe BYX Exchange - Nasdaq Global Select Market,GCY
8,[EQUITY],Apple Ord Shs APPLE ORD APPLE INC,AAPL.PH,NASDAQ OMX PHLX Global Select Market,GSX
9,[EQUITY],Apple Ord Shs APPLE ORD,AAPL.DY,Cboe EDGA Exchange - Nasdaq Global Select Market,GDA


#### Example - Vessels
List all Very Large Gas Containers (VLGC) that are in service within the last week.

In [3]:
# Build date range expression for the location of the vessel for the last 7 days
date_7_days_ago = dt.datetime.today() - dateutil.relativedelta.relativedelta(days=7)
date_range = f'LocationTimestamp ge {date_7_days_ago.strftime("%Y-%m-%d")} and \
               LocationTimestamp lt {dt.datetime.today().strftime("%Y-%m-%d")}'

In [13]:
# Show the location of each vessel and some general details such as location, destination and coordinates.
rdp.search(
    view = rdp.SearchViews.VesselPhysicalAssets,
    filter = f"RCSAssetTypeLeaf eq 'VLGC' and VesselStatus eq 'IN SERVICE' and {date_range}",
    select = 'RIC, SynonymSearchAll, DTSource, LocationTimestamp, IMO,Latitude, Longitude, LocationDestination',
    top=100
)

,Longitude,LocationTimestamp,DTSource,Latitude,SynonymSearchAll,RIC,IMO,LocationDestination
0,33.153282,2020-12-09T07:47:24.000Z,Red Sea|Suez Canal Tanker Zone|Egypt,28.742132,Egyptian Liquefied Petroleum Gas Very Large Gas Carrier,C}KY7309491970,9350288,WADI FEIRAN [Wadi Feiran Oil Terminal]
1,113.839073,2020-12-09T21:08:19.000Z,Guangdong|South China Sea|Canton Tanker Zone|China (Mainland),22.497496,Chinese Liquefied Petroleum Gas Very Large Gas Carrier,C}KY7309636854,9719501,CN SHK [Shekou]
2,130.100449,2020-12-09T13:59:15.000Z,East China Sea|Japan Tanker Zone,31.010271,Chinese Japanese Liquefied Petroleum Gas Very Large Gas Carrier Korean,C}KY7309843620,9759185,KR TSN [Daesan]
3,-97.569687,2020-12-08T23:00:18.000Z,Pacific Ocean (Central America)|West Mexico Tanker Zone,14.309810,Mexican Korean Liquefied Petroleum Gas Very Large Gas Carrier,C}KY7309844039,9797864,PABLB [Balboa]
4,66.395981,2020-12-08T18:31:51.000Z,Arabia Sea,18.539083,Liquefied Petroleum Gas Very Large Gas Carrier,C}KY7309451481,8519966,IN DMQ
5,72.558334,2020-12-08T03:28:56.000Z,Arabia Sea|West Coast India Tanker Zone,19.107578,Indian Kuwaiti Liquefied Petroleum Gas Very Large Gas Carrier,C}KL7309478067,9172636,INBOM
6,-0.345000,2020-12-08T06:52:52.000Z,West Med|West Med Tanker Zone,36.145489,Algerian Liquefied Petroleum Gas Very Large Gas Carrier,C}KY7309478328,9176369,PTSET->DZAZW [Arzew]
7,50.810871,2020-12-08T05:21:55.000Z,Gulf of Oman|Arabian Gulf Tanker Zone,27.123461,Indian Liquefied Petroleum Gas Very Large Gas Carrier,C}KY7309489467,9320740,IN DAH>SA RTA
8,104.460350,2020-12-09T18:01:58.000Z,Java Sea|Singapore Tanker Zone,1.385350,Chinese Liquefied Petroleum Gas Very Large Gas Carrier Qatari,C}KY7309492522,9356892,NINGBO CHINA [Ningbo and Zhoushan]
9,87.088913,2020-12-09T23:30:18.000Z,Bay of Bengal|Chittagong Tanker Zone,19.090334,Indian Liquefied Petroleum Gas Very Large Gas Carrier,C}KY7309500853,9454010,FUJAIRAH [Fujairah]


#### Example - Floating Rate Notes
Retrieve all floating rate notes (floating coupon) in the healthcare sector which are issued in Euros.  For each, ensure the coupon rate is great than 2 and our Moody's rating is greater than 'Baa1'.

To determine the rating, we apply the following logic utilizing the RatingsScope property - which is a nested property providing additional details:

- RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10)

The RatingType of 'MIS' refers to the "Moody's Long-term Issuer Rating".
The CurrentRatingRank < 10 instructs search to look for ratings less than 10 ('Baa1').  **Note**: The lower the rank, the better the rating.

In [14]:
rdp.search(
    view = rdp.SearchViews.GovCorpInstruments,
    filter = "CouponRate gt 2 and \
              RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10) and \
              RCSCouponTypeName eq 'Floating Coupon' and \
              IsActive eq true and \
              RCSTRBC2012Name eq 'Healthcare' and \
              RCSCurrencyLeaf xeq 'Euro'",
    select = "CouponCurrency, IssuerName, IssueDate, RIC, CouponRate, ISIN, Ticker, RCSTRBC2012Leaf, MaturityDate, AssetState",   
)

,IssuerName,ISIN,CouponCurrency,RCSTRBC2012Leaf,Ticker,RIC,IssueDate,CouponRate
0,Sanofi FR,FR0000140022,EUR,Pharmaceuticals (NEC),SASY,FR0000140022=,1983-07-01T00:00:00.000Z,15.5
1,Sanofi FR,FR0000047656,EUR,Pharmaceuticals (NEC),SASY,AVEPpaEUR.MIp,1987-10-01T00:00:00.000Z,15.5


#### Example - Outstanding Green Bonds
List the outstanding Green Bonds for Telefonica.

In [15]:
rdp.search(
    view = rdp.SearchViews.GovCorpInstruments,
    filter = "IssuerName eq 'Telefonica' and IsGreenBond eq true",
    select = "IssuerName, DocumentTitle, RIC, CouponRate, ISIN, IssuerTicker, DebtTypeDescription, EOMAmountOutstanding, FaceOutstanding"
)

,DocumentTitle,ISIN,IssuerName,IssuerTicker,DebtTypeDescription,FaceOutstanding,RIC,CouponRate,EOMAmountOutstanding
0,"Telefonica Europe BV, Bond, TEF 2.502",XS2109819859,Telefonica Europ,TEFBV,Junior Subordinated Note,500000000,ES210981985=,2.502,500000000
1,"Telefonica Emisiones SAU, Plain Vanilla Fixed Coupon Bond, TEF 1.069 05-Feb-2024",XS1946004451,Telefonica Emis,TEFEMI,Senior Note,1000000000,ES194600445=,1.069,1000000000


#### Example - Equity Index listing in the U.S.A

In [16]:
# Based on the Asset Category (Equity Index), list the top 10 indexes within the USA.
rdp.search(
    view = rdp.SearchViews.IndexInstruments,
    filter = "RCSIndexCountryGroupLeaf xeq 'United States' and RCSAssetCategoryLeaf xeq 'Equity Index'",
    select = "DocumentTitle, RIC, RCSTRBC2012Leaf"
)

,RCSTRBC2012Leaf,DocumentTitle,RIC
0,Trade & Business Publishing,"Dow Jones Industrial Average Index, Equity Index",.DJI
1,Magazine Publishing,"S&P 500 Index - CBOE, Equity Index",.SPX
2,UK Investment Trusts,"NASDAQ 100 Index, Equity Index",.NDX
3,Financial & Commodity Market Operators & Service Providers (NEC),"NASDAQ Composite Index, Equity Index",.IXIC
4,Professional Information Services (NEC),"Refinitiv United States Price Return Index, Equity Index",.TRXFLDUSP
5,Trade & Business Publishing,"Dow Jones Transport Average Index, Equity Index",.DJT
6,Trade & Business Publishing,"Dow Jones Composite Index, Equity Index",.DJA
7,Trade & Business Publishing,"Dow Jones Utility Average Index, Equity Index",.DJU
8,Magazine Publishing,"S&P 600 Small Cap Index, Equity Index",.SPCY
9,Magazine Publishing,"S&P 500 Index - CME, Equity Index",.INX


#### Example - Search for Brazil global sovereign bonds in USD with a duration greater than 1

In [17]:
rdp.search(
    view=rdp.SearchViews.FixedIncomeInstruments,
    filter = "RCSIssuerCountryLeaf eq 'Brazil' and \
              RCSCurrencyLeaf eq 'US Dollar' and \
              RCSTRBC2012Leaf eq 'Government & Government Finance' and \
              OptionAdjustedDuration gt 1",
    select ="RIC, ISIN, MaturityDate, CouponRate, IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration"
)

,IndustrySector,ISIN,FaceOutstandingUSD,OptionAdjustedDuration,RIC,CouponRate,MaturityDate
0,SOVERGRN,US105756BU30,2150000000,1.995180,105756BU3=,2.625,2023-01-05T00:00:00.000Z
1,SOVERGRN,US105756AR10,1029030000,2.959345,BR012631375=,8.875,2024-04-15T00:00:00.000Z
2,SOVERGRN,US105756AZ36,85557000,2.959345,BR017244388=,8.875,2024-04-15T00:00:00.000Z
3,SOVERGRN,US105756BV13,4300000000,3.698457,105756BV1=,4.250,2025-01-07T00:00:00.000Z
4,SOVERGRN,US105756BF62,688201000,3.514352,BR021210641=,8.750,2025-02-04T00:00:00.000Z
5,SOVERGRN,US105756CD06,1750000000,4.199081,105756CD0=,2.875,2025-06-06T00:00:00.000Z
6,SOVERGRN,US105756BX78,2176245000,4.608230,105756BX7=,6.000,2026-04-07T00:00:00.000Z
7,SOVERGRN,US105756AE07,765023000,5.052952,105756AE0=,10.125,2027-05-15T00:00:00.000Z
8,SOVERGRN,US105756BZ27,3000000000,5.823233,105756BZ2=,4.625,2028-01-13T00:00:00.000Z
9,SOVERGRN,US105756CA66,2000000000,6.928336,105756CA6=,4.500,2029-05-30T00:00:00.000Z


In [20]:
# Alternatively, you have the option of trying it this way which will render the same results
rdp.search(
    view=rdp.SearchViews.FixedIncomeInstruments,
    filter = "IssuerCountryName eq 'Brazil' and \
              OptionAdjustedDuration gt 1 and \
              Currency eq 'USD' and IndustrySectorDescription eq 'Sovereign'",
    select ="RIC, ISIN, MaturityDate, CouponRate,IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration"
)

,IndustrySector,ISIN,FaceOutstandingUSD,OptionAdjustedDuration,RIC,CouponRate,MaturityDate
0,SOVERGRN,US105756BU30,2150000000,1.995180,105756BU3=,2.625,2023-01-05T00:00:00.000Z
1,SOVERGRN,US105756AR10,1029030000,2.959345,BR012631375=,8.875,2024-04-15T00:00:00.000Z
2,SOVERGRN,US105756AZ36,85557000,2.959345,BR017244388=,8.875,2024-04-15T00:00:00.000Z
3,SOVERGRN,US105756BV13,4300000000,3.698457,105756BV1=,4.250,2025-01-07T00:00:00.000Z
4,SOVERGRN,US105756BF62,688201000,3.514352,BR021210641=,8.750,2025-02-04T00:00:00.000Z
5,SOVERGRN,US105756CD06,1750000000,4.199081,105756CD0=,2.875,2025-06-06T00:00:00.000Z
6,SOVERGRN,US105756BX78,2176245000,4.608230,105756BX7=,6.000,2026-04-07T00:00:00.000Z
7,SOVERGRN,US105756AE07,765023000,5.052952,105756AE0=,10.125,2027-05-15T00:00:00.000Z
8,SOVERGRN,US105756BZ27,3000000000,5.823233,105756BZ2=,4.625,2028-01-13T00:00:00.000Z
9,SOVERGRN,US105756CA66,2000000000,6.928336,105756CA6=,4.500,2029-05-30T00:00:00.000Z


#### Example - Top executives

In [23]:
# Using the People View, we can utilize the OfficerDirector property, which contains a number of nested or sub-properties 
# that allow me to perform a more fine-grained search against active people that have been assigned a title code of 
# CEO (Chief Executive Officer). The results are listed based on a default rank defined by the search service.
rdp.search(
    view = rdp.SearchViews.People,
    filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')",
    top = 20,
    select = "FullName, PrimaryTitleAsReported, PrimaryOfficerDirector, RIC, OfficerDirectorActiveCompanies, PrimaryOfficerDirectorRIC"
)

,PrimaryOfficerDirector,FullName,PrimaryOfficerDirectorRIC,PrimaryTitleAsReported,OfficerDirectorActiveCompanies
0,Microsoft Corp,John Wendell Thompson,MSFT.O,Independent Non-Executive Chairman of the Board,"[User Replay Ltd, Crystal Decisions Inc, Commerce Decisions Ltd, Illumina Inc, Microsoft Corp]"
1,Microsoft Corp,Satya Nadella,MSFT.O,"Chief Executive Officer, Director","[Starbucks Corp, Chandler Insurance Company Ltd, Microsoft Corp]"
2,Apple Inc,Art D. Levinson,AAPL.O,Independent Chairman of the Board,"[Apple Inc, Genentech Inc]"
3,Apple Inc,Tim D. Cook,AAPL.O,"Chief Executive Officer, Director","[Apple Inc, Nike Inc]"
4,Amazon.com Inc,Jeff P. Bezos,AMZN.O,"Chairman of the Board, President, Chief Executive Officer","[Institute For Advanced Study, Amazon.com Inc]"
5,Amazon.com Inc,Keith Brian Alexander,AMZN.O,Director,"[Amazon.com Inc, Ironnet Cybersecurity Inc]"
6,Amazon.com Inc,Jon J. Rubinstein,AMZN.O,Independent Director,"[Palm Inc, Amazon.com Inc]"
7,Alphabet Inc,Sundar Pichai,GOOGL.O,"Chief Executive Officer, Director","[Alphabet Inc, Ruba Inc (Fka Kudo Inc)]"
8,Alphabet Inc,Roger W. Ferguson,GOOGL.O,Independent Director,"[Teachers Insurance and Annuity Association of America, Audax Health Solutions LLC, Alphabet Inc, International Flavors & Fragrances Inc..."
9,Facebook Inc,Mark Zuckerberg,FB.O,"Chairman of the Board, Chief Executive Officer, Founder",[Facebook Inc]


#### Example - Live Options
The following example will specify an underlying instrument (RIC) and pull the list of options that are active, i.e. not expired.

In [7]:
# Define our underlying
underlying = "'AAPL.O'"

# To search for our instrument, I'm utilizing the property: 'UnderlyingQuoteRIC'.  The property does not support an 
# exact match 'xeq', so we must use the partial match 'eq' to find all hits. Depending on the instrument you specify, 
# this may introduce additional hits.  For example, if you are using the underlying 'AAPL.O', search will also find a 
# hit on 'AAPL.OQ', for example. To protect against this, we can utilize a combination of the built-in functions, 
# 'startswith' and 'endwith' to perform our match.
rdp.search(
    view = rdp.SearchViews.EquityDerivativeQuotes,
    filter = "AssetState eq 'AC' and \
              startswith(UnderlyingQuoteRIC, " + underlying + ") and \
              endswith(UnderlyingQuoteRIC," + underlying + ") and \
              RCSAssetClass eq 'OPT' and IsChain eq false and \
              ExpiryDate ge " + dt.datetime.today().strftime('%Y-%m-%d'),
    select = "DocumentTitle, RIC, ExpiryDate, CallPutOption, StrikePrice",
    top = 10000
)

,DocumentTitle,CallPutOption,StrikePrice,RIC,ExpiryDate
0,"OPRA Apple Equity Option 120 Put Feb 2021 , Equity Cash Option, Put 120 USD 19-Feb-2021, OPRA",Put,120.0,AAPLN192112000.U,2021-02-19T00:00:00.000Z
1,"OPRA Apple Equity Option 130 Call Feb 2021 , Equity Cash Option, Call 130 USD 19-Feb-2021, OPRA",Call,130.0,AAPLB192113000.U,2021-02-19T00:00:00.000Z
2,"OPRA Apple Equity Option 125 Call Feb 2021 , Equity Cash Option, Call 125 USD 19-Feb-2021, OPRA",Call,125.0,AAPLB192112500.U,2021-02-19T00:00:00.000Z
3,"OPRA Apple Equity Option 130 Put Feb 2021 , Equity Cash Option, Put 130 USD 19-Feb-2021, OPRA",Put,130.0,AAPLN192113000.U,2021-02-19T00:00:00.000Z
4,"OPRA Apple Equity Option 120 Call Feb 2021 , Equity Cash Option, Call 120 USD 19-Feb-2021, OPRA",Call,120.0,AAPLB192112000.U,2021-02-19T00:00:00.000Z
...,...,...,...,...,...
2151,"OPRA Apple Equity Option 143 Put Mar 2021 , Equity Cash Option, Put 143 USD 05-Mar-2021, OPRA",Put,143.0,AAPLO052114300.U,2021-03-05T00:00:00.000Z
2152,"OPRA Apple Equity Option 129 Call Mar 2021 , Equity Cash Option, Call 129 USD 05-Mar-2021, OPRA",Call,129.0,AAPLC052112900.U,2021-03-05T00:00:00.000Z
2153,"OPRA Apple Equity Option 125 Put Mar 2021 , Equity Cash Option, Put 125 USD 05-Mar-2021, OPRA",Put,125.0,AAPLO052112500.U,2021-03-05T00:00:00.000Z
2154,"OPRA Apple Equity Option 128 Put Mar 2021 , Equity Cash Option, Put 128 USD 05-Mar-2021, OPRA",Put,128.0,AAPLO052112800.U,2021-03-05T00:00:00.000Z
